In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [209]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
    
countries = ["Argentina", "Australia","Austria","Belgium","Bulgaria", "Canada","Chile","China",
             "Costa Rica","Croatia","Czech Republic", "Denmark","Estonia","Finland","France",
             "Germany", "Greece","Hungary","India","Ireland","Israel","Italy", "Japan","Latvia",
             "Lithuania","Luxembourg","Malta","Monaco", "Netherlands","New Zealand","Norway",
             "Oman","Poland","Portugal","Romania", "Russia","Singapore","Slovakia","South Africa",
             "South Korea", "Spain","Sweden","Switzerland","UAE","United Kingdom","USA"]

base_url = 'https://theoperacritic.com/searchsuite/performance.php?opera=&city=&sday=01&smth=01&syr=2000&eday=00&emth=00&eyr=0000&searchtype=3&x=36&y=13&country='
        
cookies = dict(
    PHPSESSID='3a6b3abf68cfc4bc98dbcf1072989270',
    __utma='65021163.1658740872.1530787545.1530831768.1530890054.5',
    __utmb='65021163.16.10.1530890054',
    __utmc='65021163',
    __utmt='1',
    __utmz='65021163.1530787545.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)',
    pword='0892332064fff5e2',
    valid_user='liev.garcia%40gmail.com'
)    

events = []
count = 0

In [ ]:
for country in countries:

    print('Starting: ' + country)
    url = base_url + country
    print(url)
    count = len(events)
    
    while url:
        r  = requests.get(url, cookies=cookies)
        soup = BeautifulSoup(r.text, "html5lib")
        listings = soup.find('div', attrs={"class": "resultsbox"}).find('tbody').find('table').find('tbody').find_all('tr')    
        url = None

        for listing in listings:

            if listing.find('a').get('class')!=['showajaxpopup']:
                if 'offset' not in listing.find_all('a')[-1].get('href'):

                    items     = listing.find_all('td')    
                    city      = items[0].get_text().strip()
                    composer  = items[1].find('h4').get_text().strip().split(':')[0].strip()            
                    work      = items[1].find('h4').get_text().strip().split(':')[-1].strip()
                    venue     = items[1].find('b').get_text().strip()
                    deets_url = listing.find('a', attrs={"class": "showajaxpopup"}).get('href')[2:]
                    deets_url = 'https://theoperacritic.com' + deets_url

                    r     = requests.get(deets_url)
                    soup  = BeautifulSoup(r.text, "html5lib") 
                    dates = soup.find('strong')
                    if dates:
                        dates = dates.parent.parent.next_sibling.next_sibling.get_text().strip()

                    event = []
                    event.append(country)    
                    event.append(city)
                    event.append(composer)            
                    event.append(work)
                    event.append(venue)                            
                    event.append(dates)                
                    event.append(deets_url)                
#                     print(event)
                    events.append(event)   
                else:
                    if listing.find_all('a')[-1].get_text().strip()=='next >>':
                        url = base_url + listing.find_all('a')[-1].get('href')

    print(len(events)-count)                        
    print('Finished: ' + country)

In [ ]:
import pandas as pd                     
headers = ['country', 'city', 'composer', 'work', 'venue', 'dates', 'dates_url']
df = pd.DataFrame(events, columns=headers)
df.to_csv('../../data/raw/operacritic.csv')

In [14]:
import re

years = []
for i in range(2000, 2020):
    years.append(str(i))

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 
        '31', '1', '2', '3', '4', '5', '6', '7', '8', '9']    
dow = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def get_dates(date_str):
    
    dates_list = []        
    dates = re.split('; |\(|\)|, | |\*|\n',date_str)
    dates = dates[::-1]
    year = ''
    month = ''
    day = ''
    
    if any(ext in dates for ext in dow):
        for idx in range(len(dates)):
            if dates[idx] in days:
                day = dates[idx]
            if dates[idx] in months:
                month = dates[idx]
            if dates[idx] in years:
                year = dates[idx]
                new_date = year + '-' + month + '-' + day
                dates_list.append(new_date)
    else:       
        for idx in range(len(dates)):
            if dates[idx] in years:
                year = dates[idx]
            if dates[idx] in months:
                month = dates[idx]
            if dates[idx] in days:
                new_date = year + '-' + month + '-' + str(dates[idx])
                dates_list.append(new_date)
    return dates_list[::-1]

In [18]:
import pandas as pd                     
df = pd.read_csv('../../data/raw/operacritic.csv', index_col=0)
# df.head()

In [19]:
#Edit a few rows for which the dates are not complete.
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=auskingi0614', 'dates'] = \
    '10, 13, 14(m), 14, 15(m), 17(m), 18(m), 18, 19, 20, 21(m), 21, 22(m), 24(m), 25(m), 25, 26, 27, 28(m), 28, 29(m)' + \
    ' June, 1(m), 2(m), 2, 3, 4, 5(m), 5, 6(m), 8(m), 9(m), 9, 10, 11, 12(m), 12, 13(m), 15(m), 16, 17, 18, 19(m), 19' + \
    ', 20(m), 22(m), 23(m), 23, 24, 25, 26(m), 26, 27(m), 29(m), 30, 31 July, 1, 2(m), 2, 3(m), 5(m), 6(m), 6, 7, 8, 9(m)' + \
    ', 9, 10(m)' + ', 11, 12, 13, 14, 15, 16, 17 August 2014'    
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=auskingi0914', 'dates'] = \
    '7(m), 7, 9, 10(m), 10, 12, 13(m), 13, 14(m), 16(m), 17(m), 17, 18, 19, 20(m), 20, 21(m), 23(m), 24(m), 24, 25,' + \
    '26, 27(m), 27, 28(m), 30(m) September, 1(m), 1, 2, 3, 4(m), 4, 5(m), 7(m), 8, 9, 10, 11(m), 11, 12(m), 14(m), ' + \
    '15(m), 15, 16, 17, 18(m), 18, 19(m), 21(m), 22(m), 22, 23, 24, 25(m), 25, 26(m), 28(m), 29(m), 29, 30, 31 ' + \
    'October, 1(m), 1 Novemb' + 'er 2014'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=ausanythi0915', 'dates'] = \
    '8, 9(m), 9, 10, 11, 12(m), 12, 13(m), 15(m), 16(m), 16, 17, 18, 19(m), 19, 20(m), 22(m), 23(m), 23, 24, 25, 26(m) ' + \
    ', 26, 27(m), 29(m), 30(m), 30 September, 1, 2, 3(m), 4(m), 6(m), 7(m), 7, 8, 9, 10(m), 10, 11(m), 13(m), 14(m), ' + \
    '14, 15, 16, 17(m), 17, 18(m), 20(m), 21(m), 21, 22, 23, 24(m), 24, 25(m), 27(m), 28(m), 28, 29(m), 29, 30,  ' + \
    '31(m), 31 October' + ' 2015'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=ausmyfair0816', 'dates'] = \
    '30, 31 August, 1, 2, 3(m), 3, 4(m), 6, 7(m), 7, 8, 9, 10(m), 10, 11(m), 13(m), 14(m), 14, 15, 16, 17(m), 17, ' + \
    '18(m), 20(m), 21(m), 21, 22, 23, 24(m), 24, 25(m), 27, 28(m), 28, 29, 30 September, 1(m), 1, 2(m), 4(m), 5(m), ' + \
    '5, 6, 7, 8(m), 8, 9(m), 11(m), 12(m), 12, 13, 14, 15(m), 15, 16(m), 18(m), 19(m), 19, 20, 21, 22(m), 22, 23, ' + \
    '25(m), 26(m), 26, 27,' + '28,29,30,31 October 1,2,3,4,5 November 2016'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=ausmyfair0517', 'dates'] = \
    '11, 12, 13(m), 13, 14(m), 16, 17(m), 17, 18, 19, 20(m), 20, 21(m), 23, 24(m), 24, 25, 27(m), 27, 28(m), 30(m), ' + \
    '31(m), 31 May, 1, 2, 3(m), 3, 4(m), 6(m), 7(m), 7, 8, 9, 10(m), 10, 11(m), 13(m), 14(m), 14, 15, 16, 17(m), 17,' + \
    '18(m), 20(m), 21(m), 21, 22, 23, 24(m), 24, 25(m), 27(m), 28(m), 28, 29, 30 June, 1(m), 1, 2(m), 4(m), 5(m), 5, ' + \
    '6, 7, 8(m), 8,' + '9, 10(m), 10, 11(m), 13(m), 14(m), 14, 15, 16, 17(m), 17, 18(m), 20(m), 21(m), 21, 22, 23, 24(m), 24' + \
    '25(m), 27(m), 28(m), 28, 29, 30 July 2017'    
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=erlring0714', 'dates'] = \
    '18, 19, 25, 26, 30 July, 1, 2(m), 2, 3(m) August 2014'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=tdcwside1012', 'dates'] = \
    '17(m), 26, 27, 28(m), 28, 30, 31 October, 1, 2, 3(m), 3, 4(m), 4, 6, 7, 8, 9, 10(m), 10, 11(m), 11, 13, 14, 15,' + \
    '16, 17(m), 17, 18(m), 18, 19, 20, 21, 22, 23, 24(m), 24, 25(m), 26, 28, 29, 30 November, 1(m), 1, 2(m), 2, 3, 4,' + \
    '5, 6, 8(m), 9(m), 9, 11, 12, 13, 14, 15(m), 15, 16(m), 16, 18, 19, 20, 21, 22(m), 22, 23(m), 23, 24, 26, 27, 28,' + \
    '29(m), 29, 3' + '0 December 2012, 1 January 2013'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=lyobrundi0316', 'dates'] = \
    '29, 30, 31 March, 1, 2(m), 2, 3 April 2016'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=kartroyen0113', 'dates'] = \
    '13, 17, 19 January, 2 February 2013'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=bhvsingin0913', 'dates'] = \
    '17, 21, 29 September, 6(m), 9, 13(m), 20 October, 21, 23(m) November, 21, 31 December 2013, 19(m) January, 7' + \
    ', 22 February, 15 March, 5 April, 4, 12 May, 1(m), 9, 28 June 2014'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=weibarbei1217', 'dates'] = \
    '12(m), 19(m), 26(m) December 2017, 9(m) January, 28(m) February 2018'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=hlkballo1215', 'dates'] = \
    '11, 12, 13, 15 December 2015, 17 February 2016'
df.loc[df['dates_url']=='https://theoperacritic.com/scripts/ScheduleDetails.php?schedule=gbghair0916', 'dates'] = \
    '10, 14, 15, 17, 18(m), 18, 21, 23, 25, 27, 29, 30 September, 1(m), 1, 5, 7, 13, 14, 15(m), 15, 19, 22, 23(m)' + \
    ', 25 October, 1, 2, 6(m), 8, 10, 11, 12(m), 12, 15, 18, 19(m), 19, 22, 24, 26 November, 1, 28, 30 December 2016' + \
    ', 13, 14(m), 14, 20, 21(m), 21, 24, 26, 27, 28(m), 28, 31 January, 1, 17, 22, 24, 25(m), 25 February, 1, ' + \
    '2, 7, 12(m), 14, 17, 21, 2' + '4, 26, 27, 28 March 2017'

In [22]:
from datetime import datetime
events=[]
issues=[]

for idx,row in df.iterrows():
    dates = get_dates(row['dates'])
    for date in dates:
        event=[]
        event.append(row['country'])
        event.append(row['city'])
        event.append(row['composer'])
        event.append(row['work'])
        event.append(row['venue'])
        event.append(datetime.strptime(date, '%Y-%B-%d').date())
        event.append(row['dates_url'])
        events.append(event)

In [24]:
import pandas as pd                     
headers = ['country', 'city', 'composer', 'work', 'venue', 'date', 'dates_url']
df = pd.DataFrame(events, columns=headers)
df.to_csv('../../data/processed/listings/operacritic.csv')

In [23]:
len(events)

103565